OBJECTIVE: SEEK TO PAIR BIASED FULL DATA WITH THE NEUTRAL DATA SET TO CREATE A VALIDATION TOOL FOR BIASED FULL.
THIS WAY A BINARY TARGET COULD BE CREATED FOR THE RECORDS IN BIASED FULL FOR A BINARY CLASSIFICATION.

Written by: Etienne Ndedi
Date: 9_27_2024

**NOTE: THIS STRATEGY IS ABANDONED AS THE NEUTRAL SUBSET ONLY CONTAINS COMMON IDs WITH THE BIASED FULL SUBSET. THE TEXT IS NOT THE SAME**

In [ ]:
import pandas as pd
import numpy as np
import csv
from sklearn import preprocessing
import spacy
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid",color_codes=True)
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.metrics import classification_report


In [ ]:
#!pip install -U spacy==3.*
#!python -m spacy download en_core_web_sm
#!pip install -m info

Reading in the full data set.
- src_raw is the source text whose bias (if it exist) we are trying to
  mitigate (to result in a neutral sequence)
- tgt_raw is the proposed repharising of the text (if it is biased)
- id is a number assigned to the record or data line
- The current data strategy is to leverage the two columns to provide
  us with a framework resulting in a two-prong approach consisting of
  doing Deep Learning binary classification modeling to tag whether the text is biased or not and using LLMs to remedy the bias if it exists.
- The stretch goal is to assess the efficacy of the LLMs regarding the
  aforementioned task by doing a re-classification again

In [ ]:

df_inputs = pd.read_csv('biased.full', sep='\t', names=["id", "src_tok", "tgt_tok", "src_raw", "tgt_raw", "src_POS_tags", "tgt_parse_tags"], on_bad_lines='skip')
print(f"Number of rows in DataFrame: {len(df_inputs)}")

df_inputs.head()

Number of rows in DataFrame: 181473


,id,src_tok,tgt_tok,src_raw,tgt_raw,src_POS_tags,tgt_parse_tags
0,258378316,"during the campaign , controversy erupted over...","during the campaign , some pointed out alleged...","during the campaign, controversy erupted over ...","during the campaign, some pointed out alleged ...",ADP DET NOUN PUNCT NOUN VERB ADP VERB NOUN ADP...,prep det pobj punct nsubj ROOT prep amod pobj ...
1,486527143,nic ##aea was con ##vo ##ked by the emperor co...,nic ##aea was con ##vo ##ked by the emperor co...,nicaea was convoked by the emperor constantine...,nicaea was convoked by the emperor constantine...,NOUN NOUN VERB VERB VERB VERB ADP DET NOUN NOU...,nsubjpass nsubjpass auxpass ROOT ROOT ROOT age...
2,54024499,it was rather unfortunate that he ve ##hem ##e...,he ve ##hem ##ently opposed the bud ##ding ind...,it was rather unfortunate that he vehemently o...,he vehemently opposed the budding indian scien...,PRON VERB ADV ADJ ADP PRON ADV ADV ADV VERB DE...,nsubj ROOT advmod acomp mark nsubj advmod advm...
3,160186886,dennis the menace is an american animated seri...,dennis the menace is an american animated seri...,dennis the menace is an american animated seri...,dennis the menace is an american animated seri...,VERB DET NOUN VERB DET ADJ VERB NOUN VERB ADP ...,csubj det dobj ROOT det amod amod attr acl age...
4,8797183,"today , on large farms , motorcycles , dogs or...","today , on large farms , motorcycles , dogs or...","today, on large farms, motorcycles, dogs or me...","today, on large farms, motorcycles, dogs or pe...",NOUN PUNCT ADP ADJ NOUN PUNCT NOUN PUNCT NOUN ...,npadvmod punct prep amod pobj punct conj punct...


-Ensure the source and target have different texts and validate that
 the file is solely for source bias and target

In [ ]:
#check if src_raw and tgt_raw have the same data for all id
df_inputs['id'].loc[df_inputs['src_raw'] == df_inputs['tgt_raw']].count()


0

In [ ]:
#check if src_raw and tgt_raw have the same data for all id
df_inputs['id'].loc[df_inputs['src_raw'] != df_inputs['tgt_raw']].count()

181473

Apparently the two fields are different throughout

Now there is an ancillary data set which contains only neutral data supposedly. Check if despite the difference in the biased full set, there are line of data that do not necessarily mean there is a bias because these lines are in the neutral set.

If there are ids in the neutral set, this may mean that despite the
difference highlighted in the biased full set, there are lines that have simply been rephrased without necessarily being flagged as biased.

The EDA at this stage consist of pairing the ids and asking:

- Are neutral records the same in the source and the target?
- If the above question is YES then it is worth using this file as
  to validate the biased full against neutral
- Use records of biased full that are found in neutral as statements
  that have no biased 0 and the ones that are not found in neutral as
  having a bias.
  

Let's read neutral

In [ ]:
df_neutral= pd.read_csv('neutral', sep='\t', names=["id", "src_tok", "tgt_tok", "src_raw", "tgt_raw", "src_POS_tags", "tgt_parse_tags"], on_bad_lines='skip')
print(f"Number of rows in DataFrame: {len(df_neutral)}")

Number of rows in DataFrame: 385527


Count of Neutral with the source and target being equal

In [ ]:
#check if src_raw and tgt_raw have the same data for all id in the neutral dataset
df_neutral['id'].loc[df_neutral['src_raw'] == df_neutral['tgt_raw']].count()

382026

In [ ]:
only_neutral = df_neutral['id'].loc[df_neutral['src_raw'] == df_neutral['tgt_raw']]
only_neutral.head()

,id
0,699417259
1,656122183
2,116089017
3,66650168
4,213253868


Are there biased full records in neutral? Pair their ids and src_raw
- merge biased full and neutral based on 'id' alone
- merge biased full and neutral based on 'id' and src_raw
- merge biased full and neutral based on 'id' and tgt_raw

In [ ]:
#pair the ids and create a temporary dataframe that hold the ids found in biased full and neutral
common_ids = df_inputs['id'][df_inputs['id'].isin(df_neutral['id'])]
print(f"Number of common ids (biased full and neutral): {len(common_ids)}")
# Create a dataframe resulting from a merge of biased full and neutral based on id and src_raw
temp_df1 = pd.merge(df_inputs[['id', 'src_raw']], df_neutral[['id', 'src_raw']], on=['id','src_raw'])
print(f"Number of rows in Merge of biased full and neutral based on id & src_raw Dataframe: {len(temp_df1)}")
# Create a dataframe resulting from a merge of biased full and neutral based on id and tgt_raw
temp_df2 = pd.merge(df_inputs[['id', 'tgt_raw']], df_neutral[['id', 'tgt_raw']], on=['id','tgt_raw'])
print(f"Number of rows in Merge of biased full and neutral based on id & tgt_raw Dataframe: {len(temp_df2)}")

Number of common ids (biased full and neutral): 132891
Number of rows in Merge of biased full and neutral based on id & src_raw Dataframe: 0
Number of rows in Merge of biased full and neutral based on id & tgt_raw Dataframe: 0


In [ ]:
df1 = df_inputs['id'].isin(df_neutral['id'])
df1.value_counts()

,count
id,
True,132891
False,48582


NO NEED TO PURSUE NEUTRAL AS A VALIDATION TOOL FOR THE BIASED FULLL DATA SET. THE OBJECTIVE WAS TO SEE IF THERE WERE COMMON SRC_RAW OR TGT_RAW RECORDS. HAD THIS BEEN THE CASE, BINARIZATION COULD HAVE HAPPENED BY JUST USING RECORDS THAT ARE IN NEUTRAL AS NEUTRAL. SO END OF THE ROAD HERE. NEUTRAL CANNOT BE CONSIDERED FOR THE ANALYSIS.
BINARIZATION WILL BE SOLELY ON SRC_RAW AND TGT_RAW IN BIASED FULL TO PROVIDE US WITH A BIANARY CLASSIFICATION PROBLEM